In [16]:
%config InLineBackend.figure_format='retina'

In [24]:
import numpy as np
import pandas as pd
import altair as alt
import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

In [19]:
def correlated_streams(n: int, mean: float, risk: float, corr: float) -> np.array:
        """
        Generates n returns streams with a given average mean and risk 
        and with an average correlation level corr 
        """
        num_samples = 10_000
        means = np.full(n, mean)
        
        corr_mat = np.full((n,n),corr, dtype=np.dtype("d"))
        np.fill_diagonal(corr_mat, 1,)
        cov_mat = corr_mat * risk**2
        
        streams = np.random.multivariate_normal(means, cov_mat, size=num_samples)
        
        return streams.T
        

In [27]:
n=5
mean, std, corr = 10, 15, 0.6
streams = correlated_streams(n, mean, std, corr)

In [28]:
streams.mean(axis=1)

array([10.00222395,  9.88422942,  9.94152682,  9.91113114,  9.94114824])

In [29]:
streams.std(axis=1)

array([14.97744151, 15.13176711, 15.06834027, 15.1321375 , 15.12785865])

In [26]:
streams

array([[ -2.51237014, -29.52319219,  39.51539455, ...,  -9.0968578 ,
         25.5230905 ,   8.8425909 ],
       [ -0.89574053,  13.93734329,  72.15486409, ...,  -9.39101375,
         70.02544873, -36.12234934],
       [-26.91537347,  58.90324544,  47.08517556, ..., -28.36420705,
        -62.08907707,  27.13298684],
       [ 34.91504831,  27.97122928, -43.54781673, ...,  68.83134022,
         11.30372425,  33.14512221],
       [-37.89306004,  17.97733555,  12.50989227, ...,  -1.50070738,
        -27.86640717,   7.47617213]])

In [30]:
np.corrcoef(streams)

array([[1.        , 0.60398834, 0.60501612, 0.59762202, 0.60245373],
       [0.60398834, 1.        , 0.60919205, 0.60012463, 0.60940807],
       [0.60501612, 0.60919205, 1.        , 0.60260022, 0.60928793],
       [0.59762202, 0.60012463, 0.60260022, 1.        , 0.60770469],
       [0.60245373, 0.60940807, 0.60928793, 0.60770469, 1.        ]])

In [31]:
def aggregate_risk(returns_streams: np.array, n:int) -> np.array:
    """
    Returns the ppoled risk (std) of the n first streams in return_streams
    """
    if len(returns_streams) < n:
        raise valueError(f"len of return_stream less than n: {n}")
    
    return (np.sum(returns_streams[:n], axis=0)/n).std()

In [41]:
max_assets = 20
assets = range(1, max_assets+1)

mean=10 # average mean return of 10%
risk_levels = range(1,15)

index = pd.MultiIndex.from_product([risk_levels,assets], 
                                   names=["risk_level","num_assets"])
simulated_data = pd.DataFrame(index=index)

for risk in risk_levels:
    for corr in np.arange(0.0,0.8,0.1):
        return_streams = correlated_streams(max_assets, mean, risk, corr)
        risk_level = np.zeros(max_assets)
        for num_assets in assets:
            risk_level[num_assets-1] = aggregate_risk(return_streams, num_assets)
        simulated_data.loc[(risk, ), round(corr,1)] = risk_level
simulated_data.columns.names=["correlation"]



In [42]:
simulated_data.query("risk_level==14")

correlation                  0.0        0.1        0.2        0.3        0.4  \
risk_level num_assets                                                          
14         1           13.940044  14.052212  14.019872  13.916254  14.113210   
           2            9.906013  10.438326  10.743339  11.323233  11.860038   
           3            8.080338   8.889536   9.548746  10.305909  10.954035   
           4            6.925624   8.019580   8.863970   9.713392  10.470868   
           5            6.205322   7.466439   8.375720   9.352210  10.186392   
           6            5.700413   7.081689   8.065595   9.119634   9.963650   
           7            5.301572   6.808721   7.826437   8.962939   9.805478   
           8            4.957155   6.573748   7.638013   8.834362   9.705645   
           9            4.676249   6.373963   7.502516   8.714214   9.611202   
           10           4.452441   6.226457   7.380717   8.629474   9.518935   
           11           4.231263   6.072638   7.288803   8.557967   9.462226   
           12           4.051661   5.954919   7.208471   8.513424   9.430737   
           13           3.879700   5.854814   7.155615   8.462985   9.399546   
           14           3.732644   5.762928   7.110110   8.417938   9.354802   
           15           3.607735   5.679461   7.062262   8.373636   9.327172   
           16           3.487146   5.609286   7.018271   8.335155   9.297207   
           17           3.381903   5.557641   6.976452   8.300284   9.268169   
           18           3.282142   5.499764   6.944114   8.270399   9.254312   
           19           3.194989   5.440902   6.913191   8.242426   9.235029   
           20           3.104969   5.383602   6.889684   8.212837   9.206590   

correlation                  0.5        0.6        0.7  
risk_level num_assets                                   
14         1           14.229461  13.913509  14.019948  
           2           12.140276  12.476484  12.905944  
           3           11.516892  11.989504  12.513196  
           4           11.105878  11.731565  12.324642  
           5           10.911455  11.574425  12.216736  
           6           10.765848  11.471063  12.127635  
           7           10.649080  11.387857  12.073691  
           8           10.586701  11.335413  12.036084  
           9           10.505008  11.277297  11.996258  
           10          10.454021  11.230156  11.970614  
           11          10.411668  11.202052  11.937447  
           12          10.363650  11.177086  11.918514  
           13          10.328625  11.162937  11.900059  
           14          10.302040  11.127073  11.889681  
           15          10.274919  11.123829  11.881871  
           16          10.249561  11.118075  11.873248  
           17          10.232999  11.106164  11.859629  
           18          10.210375  11.100106  11.854694  
           19          10.195172  11.089466  11.844161  
           20          10.183389  11.078735  11.833069

In [60]:
def plot_risk_level(data:np.array, risk_level:int):
    subset = data.query(f"risk_level=={risk_level}")
    stacked = subset.stack().reset_index(name='risk')
    stacked.head()
    
    chart = alt.Chart(data=stacked)
    
    highlight = alt.selection(type='single',on='mouseover',
                                   fields=['correlation'], nearest=True)
    
    base = chart.encode(
        alt.X("num_assets", axis=alt.Axis(title="Number of Assets")),
        alt.Y("risk", axis=alt.Axis(title="Risk[%]")),
        alt.Color("correlation:N", scale=alt.Scale(scheme='set2'))
        )
    points = base.mark_circle().encode(
        opacity=alt.value(0)
        ).add_selection(
        highlight
    ).properties(
        height=400,
        width=600,
        title="Risk % by number of assets in portfolio"
    )
    lines = base.mark_line().encode(
        size=alt.condition(~highlight,alt.value(1),alt.value(3)),
    tooltip=["correlation"]
    )
    return points+lines

In [61]:
plot_risk_level(simulated_data, 10)

alt.LayerChart(...)